# Хакатон Моторика

## Часть 4 Анализ данных и разрабоотка свертки 

Для улучшения качества работы наших протезов мы решаем задачу “распознавания жестов”. 

Задача: построение классификационной модели, которая по показаниям от датчиков будет определять жест, выполненный пользователем протеза во время записи данных, что отражено в значении целевой переменной.

Данные: это оптомиография, сырые значения оптической плотности (optical density) + шумы и дрейфы. Оптическая плотность может изменяться в зависимости от выполняемого жеста. Измеряется в значениях АЦП, это отчёты от 0 до 4096, в диапазоне от 0 до 3.3в. Частота измерения - 30 Гц. 

В предыдущих частях было рассмотрено создание MVP, создание малой модели на основе площади ступеньки. В этой части я хочу дорабоать малую модель. 


In [48]:
import numpy as np
import pandas as pd
# import time
import matplotlib.pyplot as plt 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [57]:
data = np.load('data/X_train.npy')
data.shape

(165, 40, 60)

In [71]:
y = pd.read_csv('data/y_train.csv', index_col=0)
y = y.Class


In [59]:
# Функция случацного добавления шума к временному ряду наблюдений с датчика
def add_gauss_noise(orig, level=3., prob=0.5):
    size = orig.shape[0]
    cond = np.where(np.random.rand(size)<=prob,1,0)
    addn = np.random.normal(loc=0.0, scale = level, size = size )
    return orig + cond * addn

In [61]:
# Функция расширения набора данных за счет добавления шума к данным
def expand_data(X_orig, y_orig, qty=165, level=3., prob=0.5):
    orig_qty = X_orig.shape[0] - 1
    orig_chan = X_orig.shape[1]
    orig_time = X_orig.shape[2]
    X_ext = X_orig ; y_ext = y_orig
    for i in range(qty):
        loc = round(np.random.rand()* orig_qty)
        y_ext = pd.concat([y_ext,pd.Series(np.array([y_orig[loc]]), name='Class') ], axis=0, ignore_index=True)
        x_add = np.zeros((orig_chan,orig_time))
        for j in range(orig_chan):
            x_add[j,:] = add_gauss_noise(X_orig[loc,j,:], level=level, prob=prob)    #.reshape(-1),spv).reshape(1,-1)
        #x_add = x_add.reshape(1,orig_chan,orig_time)
        X_ext = np.concatenate((X_ext,x_add.reshape(1,orig_chan,orig_time)),axis=0)
    return X_ext, y_ext 



Расширяю набор данных за счет дублирования старых с небольшими случайными изменениями. Идея решения позаимствована из компьютерного зрения, задачи аугментации исходных изображений.  Изменение происходит за счет добавления гауссова шума с СКО 3 и вероятностью изменения 0.5.  Уровень 3 определен в части 2 как среднее СКО уровня фонового шума с сенсоров. 

In [72]:
X_exp, y_exp = expand_data(data,y)
print(X_exp.shape, y_exp.shape)

(330, 40, 60) (330,)


In [63]:
def find_low_sens(data, level=200.):
    sens2del = list()
    for i in range(data.shape[1]):
        if data[:,i,:].max() < level:  sens2del.append(i)
    return sens2del

In [73]:
def delete_low_sens(data, sens_list):
    return np.delete(data, sens_list, axis=1)

Нахожу а затем удаляю сигналы с сенсоров, которые имеют уровень меньше 200. Уровень 200 получен в предыдущей части в работе другого участника команды. 

In [74]:
low_sens_nums = find_low_sens(X_exp, level=200.)
print('Sens to delete', low_sens_nums)
X_cleared = delete_low_sens(X_exp, low_sens_nums)
X_cleared.shape

Sens to delete [1, 3, 4, 6, 7, 9, 10, 11, 13, 14, 16, 18, 20, 22, 23, 25, 26, 28, 31, 32, 34, 35, 37, 39]


(330, 16, 60)

In [75]:
# Фильтрация первой производной по амлитуде (больше порога) и последующее интегрирование
# data - оригинальные измерения с датчиков
# thr - пороговый уровень фильтрации первой производной
def cum_signed_diff(data, thr): 
    # t_frame = data.shape[0]
    n_sens = data.shape[0]
    cdiff = np.zeros(n_sens)
    for s in range(n_sens):
        ddata = np.diff(data[s,:])
        fddata = np.where(np.abs(ddata)<thr,0,ddata)
        cdiff[s] = fddata.sum()
    return cdiff

In [76]:
# Функция расчета площади шага кривой 
# Сигнал с датчика может пойти вверх или вниз или подпрыгнуть на месте
def parse_data(data, thr):
    n_samples = data.shape[0]
    n_sens = data.shape[1]
    p_data = np.zeros((n_samples,n_sens))  # Я предполагаю сократить размерность задачи на одно измерение. 
    for i in range(n_samples):
        p_data[i] = cum_signed_diff(data[i,:,:], thr)
    return p_data

Разбираю сигнал каждого датчика - беру первую производную, фильтрую абсолютное занчение амлитуды по уровню thr = 10 (все что меньше = 0), затем рассчитываю кумулятивную сумму отфильтрованных значений первой производной - сигнал шага уровня плотности вверх или вниз.  Уровень thr = 10  определен в части 2. 

In [78]:
X_parsed = parse_data(X_cleared, 10.)
print('Размер обработанного массива значений шагов', X_parsed.shape)
print(X_parsed[0,:])

Размер обработанного массива значений шагов (330, 16)
[ -424. -1780. -1479.  -451.    32.   -15.   170.   154.    26.   228.
   -26.    61.    32.    13.   192.    72.]


In [79]:
print('Max: ', end='')
for i in range(X_parsed.shape[1]):
    print(round(X_parsed[:,i].max()), ' ', end='')

print('\nMin: ', end='')
for i in range(X_parsed.shape[1]):
    print(round(X_parsed[:,i].min()), ' ', end='')


Max: 898  1670  1482  715  674  496  396  1182  378  682  1252  907  2067  148  455  550  
Min: -1177  -1780  -1479  -775  -730  -433  -313  -932  -236  -638  -1068  -698  -2120  -141  -342  -475  

In [80]:
def find_norm_factor(data):
    n_sens = data.shape[1]
    sens_fact = np.zeros((n_sens))
    for i in range(n_sens):
        sens_fact[i] = np.abs(data[:,i]).max()
    return sens_fact

In [81]:
def normalise_sens(data, factor):
    n_sens = data.shape[1]
    for i in range(n_sens):
        data[:,i] = data[:,i] / factor[i]
    return data

Нормирую амлитуду суммарной отфильтрованной производной в каждом канале. 

In [82]:
norm_factor = find_norm_factor(X_parsed)
X_norm = normalise_sens(X_parsed,norm_factor)
print('Max: ', end='')
for i in range(X_norm.shape[1]):
    print(round(X_norm[:,i].max(),3), ' ', end='')

print('\nMin: ', end='')
for i in range(X_norm.shape[1]):
    print(round(X_norm[:,i].min(),3), ' ', end='')

Max: 0.763  0.938  1.0  0.923  0.923  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.975  1.0  1.0  1.0  
Min: -1.0  -1.0  -0.998  -1.0  -1.0  -0.873  -0.789  -0.788  -0.625  -0.936  -0.853  -0.77  -1.0  -0.952  -0.752  -0.864  

In [83]:
print(X_norm[0,:])

[-0.36023789 -1.         -0.99797571 -0.58193548  0.04382144 -0.03024194
  0.42929293  0.13028765  0.06878307  0.33424596 -0.02076677  0.06727855
  0.01509434  0.08793876  0.42197802  0.13094305]


Делаю полиномиальные признаки второго порядка. 

In [85]:
poly = PolynomialFeatures(2, include_bias=False)
X_poly = poly.fit_transform(X_norm)
X_poly.shape
print(X_poly[0,:])

[-3.60237893e-01 -1.00000000e+00 -9.97975709e-01 -5.81935484e-01
  4.38214444e-02 -3.02419355e-02  4.29292929e-01  1.30287648e-01
  6.87830688e-02  3.34245959e-01 -2.07667732e-02  6.72785496e-02
  1.50943396e-02  8.79387617e-02  4.21978022e-01  1.30943052e-01
  1.29771340e-01  3.60237893e-01  3.59508666e-01  2.09635213e-01
 -1.57861448e-02  1.08942911e-02 -1.54647580e-01 -4.69345478e-02
 -2.47782678e-02 -1.20408060e-01  7.48097861e-03 -2.42362829e-02
 -5.43755310e-03 -3.16788742e-02 -1.52012474e-01 -4.71706493e-02
  1.00000000e+00  9.97975709e-01  5.81935484e-01 -4.38214444e-02
  3.02419355e-02 -4.29292929e-01 -1.30287648e-01 -6.87830688e-02
 -3.34245959e-01  2.07667732e-02 -6.72785496e-02 -1.50943396e-02
 -8.79387617e-02 -4.21978022e-01 -1.30943052e-01  9.95955515e-01
  5.80757477e-01 -4.37327370e-02  3.01807170e-02 -4.28423915e-01
 -1.30023908e-01 -6.86438318e-02 -3.33569347e-01  2.07247352e-02
 -6.71423582e-02 -1.50637843e-02 -8.77607480e-02 -4.21123815e-01
 -1.30677985e-01  3.38648

In [86]:
# Разделение данных на обучение и валидацию
X_train, X_test, y_train, y_test = train_test_split(X_poly, y_exp, test_size = 0.2)
print('Test:', X_test.shape)

Test: (66, 152)


Пробую логистическую мультиклассовую классификацию.

In [88]:
LR_clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=10, random_state=42, max_iter=5000)
LR_clf.fit(X_train, y_train)

#y_pred = clf.predict(X_test_review_tfidf)
y_pred = LR_clf.predict(X_test)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
LR_clf.fit(X_poly,y_exp)

Test Accuracy:  1.0


LogisticRegression(C=10, max_iter=5000, multi_class='multinomial',
                   random_state=42)

Пробую собрать файл для scoreboard

In [90]:
X_score = np.load('data/X_test.npy')
print('Loaded:', X_score.shape)
X_score = delete_low_sens(X_score, low_sens_nums)
print('Deleted:', X_score.shape)
X_score = parse_data(X_score, 10.)
print('Stepped', X_score.shape)
print(X_score[0,:])
X_score = normalise_sens(X_score,norm_factor)
print(X_score[0,:])
X_score = poly.transform(X_score)
print('Polynomized', X_score.shape)


Loaded: (166, 40, 60)
Deleted: (166, 16, 60)
Stepped (166, 16)
[ 255. 1081.  867.  140. -148.  -18. -199. -116.  -32.  -93.  -52. -151.
   61.  -36. -203.  -62.]
[ 0.21665251  0.60730337  0.58502024  0.18064516 -0.20267418 -0.03629032
 -0.50252525 -0.09813875 -0.08465608 -0.13633717 -0.04153355 -0.16654198
  0.02877358 -0.24352272 -0.44615385 -0.11275652]
Polynomized (166, 152)


In [91]:
y_score = pd.Series(data=LR_clf.predict(X_score),index=np.arange(166)+165,name='Class')
y_score.to_csv('220911_01.csv', index_label='Id')

Kaggle Score: 0.98795

Результат чуточку хуже идеального.  

Пробую собрать обучающий набор из обучающих и тестовых данных и использовать ранее использованный результат для разметки данных.  Подход отчасти напомниает целевое кодирование.  Использование подхода обусловлено следующими соображениями:
- есть хорошо распознанные данные, однако модель их распознвания не влезет в контроллер; 
- можно попробовать обучить модель меньшего размера на исходных данных и результате распознавания такой большой модели. 

In [108]:

X_new = np.concatenate((np.load('data/X_train.npy'),np.load('data/X_test.npy')),axis=0)
X_new.shape

(331, 40, 60)

In [109]:
y1 = pd.read_csv('data/y_train.csv', index_col=0)
y1 = y1.Class.values
y2 = pd.read_csv('data/LR_submission121.csv', index_col=0)
y2 = y2.Class.values
y_new = np.concatenate((y1,y2),axis=0)
y_new.shape

(331,)

In [110]:
X_new, y_new = expand_data(X_new, pd.Series(y_new, name='Class'), qty=331)
print(X_new.shape, y_new.shape)

(662, 40, 60) (662,)


In [111]:
def prepare_data(X_new, low_sens_nums, norm_factor, fit=True):
    if fit : low_sens_nums = find_low_sens(X_new, level=200.)
    print('Sens to delete', low_sens_nums)
    X_new = delete_low_sens(X_new, low_sens_nums)
    print('Deleted:', X_new.shape)
    X_new = parse_data(X_new, 10.)
    print('Stepped', X_new.shape)
    print(X_new[0,:])
    if fit : norm_factor = find_norm_factor(X_new)
    X_new = normalise_sens(X_new,norm_factor)
    print(X_new[0,:])
    X_new = poly.transform(X_new)
    print('Polynomized', X_new.shape)
    return X_new, low_sens_nums, norm_factor

In [112]:
# подготовка данных и запоминание констант обработки
X_new, low_sens_nums, norm_factor = prepare_data(X_new, low_sens_nums, norm_factor, fit=True)
# Разделение на обучение и валидацию 
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2)
print('Test:', X_test.shape)
LR_clf.fit(X_train, y_train)

#y_pred = clf.predict(X_test_review_tfidf)
y_pred = LR_clf.predict(X_test)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
# Обучаю модель еще раз на всех данных
LR_clf.fit(X_new,y_new)


Sens to delete [1, 3, 4, 6, 7, 9, 10, 11, 13, 14, 16, 18, 20, 22, 23, 25, 26, 28, 31, 32, 34, 35, 37, 39]
Deleted: (662, 16, 60)
Stepped (662, 16)
[ -424. -1780. -1479.  -451.    32.   -15.   170.   154.    26.   228.
   -26.    61.    32.    13.   192.    72.]
[-0.35743609 -0.98779134 -0.96976602 -0.58193548  0.04351242 -0.02914835
  0.39945826  0.1165571   0.06878307  0.33137174 -0.02044325  0.06745177
  0.01494629  0.0785088   0.42197802  0.1302006 ]
Polynomized (662, 152)
Test: (133, 152)
Test Accuracy:  1.0


LogisticRegression(C=10, max_iter=5000, multi_class='multinomial',
                   random_state=42)

In [113]:
X_score = np.load('data/X_test.npy')
X_score, _a, _b = prepare_data(X_score, low_sens_nums, norm_factor, fit=False)
y_score = pd.Series(data=LR_clf.predict(X_score),index=np.arange(166)+165,name='Class')
y_score.to_csv('220912_01.csv', index_label='Id')

Sens to delete [1, 3, 4, 6, 7, 9, 10, 11, 13, 14, 16, 18, 20, 22, 23, 25, 26, 28, 31, 32, 34, 35, 37, 39]
Deleted: (166, 16, 60)
Stepped (166, 16)
[ 255. 1081.  867.  140. -148.  -18. -199. -116.  -32.  -93.  -52. -151.
   61.  -36. -203.  -62.]
[ 0.21496746  0.59988901  0.56848353  0.18064516 -0.20124496 -0.03497802
 -0.46760114 -0.08779626 -0.08465608 -0.13516479 -0.04088651 -0.16697078
  0.02849136 -0.21740899 -0.44615385 -0.11211718]
Polynomized (166, 152)


Kaggle Score: 1.000 

Резюме - относительно простая модель может быть дополнительно обучена на ранее не распознававшихся данных, размеченных результатами работы большей по размеру модели. 